In [ ]:
from glob import glob
import cv2
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import cross_validate, KFold, cross_val_predict, train_test_split
import tensorflow as tf
from tensorflow import keras

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
images_path = {}
images_path["Covid"] = glob("/content/drive/MyDrive/X-Ray Image DataSet/Covid-19/*.png")
images_path["Covid"] += glob("/content/drive/MyDrive/X-Ray Image DataSet/Covid-19/*.jpeg")
images_path["Covid"] += glob("/content/drive/MyDrive/X-Ray Image DataSet/Covid-19/*.jpg")

images_path["NoFindings"] = glob("/content/drive/MyDrive/X-Ray Image DataSet/No_findings/*.png")
images_path["NoFindings"] += glob("/content/drive/MyDrive/X-Ray Image DataSet/No_findings/*.jpeg")
images_path["NoFindings"] += glob("/content/drive/MyDrive/X-Ray Image DataSet/No_findings/*.jpg")

images_path["Pneumonia"] = glob("/content/drive/MyDrive/X-Ray Image DataSet/Pneumonia/*.png")
images_path["Pneumonia"] += glob("/content/drive/MyDrive/X-Ray Image DataSet/Pneumonia/*.jpeg")
images_path["Pneumonia"] += glob("/content/drive/MyDrive/X-Ray Image DataSet/Pneumonia/*.jpg")
images_class = {
    "Covid": 0,
    "Pneumonia": 1,
    "NoFindings": 2
}

In [ ]:
X = []
Y = []

for label in images_path:
    for image_path in images_path[label]:
        image = cv2.imread(image_path)
        image = cv2.resize(image,(224, 224))
        X.append(image)
        Y.append(images_class[label])

In [ ]:
X = np.array(X)
Y = np.array(Y)


In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(X, Y):
    X_train = X[train_index]
    X_test = X[test_index]
    y_train=Y[train_index]
    y_test=Y[test_index]

In [ ]:
X_train=np.array(X_train).reshape(-1,224,224,3).astype('float32')
y_train=np.array(y_train)
X_test=np.array(X_test).reshape(-1,224,224,3).astype('float32')
y_test=np.array(y_test)

In [ ]:
X_train=keras.applications.vgg16.preprocess_input(X_train)
X_test=keras.applications.vgg16.preprocess_input(X_test)

In [ ]:
vgg=keras.applications.vgg16.VGG16(input_shape=(224,224,3),weights='imagenet',include_top=False)

58900480/58889256 [==============================] - 2s 0us/step


In [ ]:
my_smaller_model =keras.models.Model(vgg.input,vgg.get_layer("block3_conv3").output)
my_smaller_model.trainable = False

In [ ]:
for layer in my_smaller_model.layers:
     layer.trainable = False

In [ ]:
base_model_output = my_smaller_model.output
gap = keras.layers.GlobalAveragePooling2D()(base_model_output)
fc = keras.layers.Dense(units=256, activation="relu")(gap)
output = keras.layers.Dense(3, activation="softmax")(fc)

In [ ]:
transfer_model = keras.models.Model(my_smaller_model.input, output)

In [ ]:
transfer_model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])

In [ ]:
transfer_model.fit(X_train,y_train,epochs=30)

Epoch 1/30
29/29 [==============================] - 12s 64ms/step - loss: 30.9570 - accuracy: 0.5756
Epoch 2/30
29/29 [==============================] - 2s 59ms/step - loss: 7.7262 - accuracy: 0.7256
Epoch 3/30
29/29 [==============================] - 2s 58ms/step - loss: 4.8090 - accuracy: 0.7500
Epoch 4/30
29/29 [==============================] - 2s 58ms/step - loss: 3.7618 - accuracy: 0.7867
Epoch 5/30
29/29 [==============================] - 2s 59ms/step - loss: 3.7156 - accuracy: 0.7722
Epoch 6/30
29/29 [==============================] - 2s 59ms/step - loss: 7.8818 - accuracy: 0.7078
Epoch 7/30
29/29 [==============================] - 2s 59ms/step - loss: 4.0859 - accuracy: 0.7700
Epoch 8/30
29/29 [==============================] - 2s 58ms/step - loss: 4.1805 - accuracy: 0.7711
Epoch 9/30
29/29 [==============================] - 2s 59ms/step - loss: 2.4937 - accuracy: 0.8233
Epoch 10/30
29/29 [==============================] - 2s 59ms/step - loss: 2.3959 - accuracy: 0.8056
Epoch 1

In [ ]:
my_smaller_model.trainable = True
for layer in my_smaller_model.layers:
     layer.trainable = True 

In [ ]:
ADAM= keras.optimizers.Adam(learning_rate=0.00001)
transfer_model.compile(loss="sparse_categorical_crossentropy",optimizer=ADAM,metrics=['accuracy'])

In [ ]:
transfer_model.fit(X_train,y_train,batch_size=10,epochs=20)

Epoch 1/20
90/90 [==============================] - 7s 60ms/step - loss: 1.2185 - accuracy: 0.8567
Epoch 2/20
90/90 [==============================] - 5s 59ms/step - loss: 1.1436 - accuracy: 0.8511
Epoch 3/20
90/90 [==============================] - 5s 59ms/step - loss: 0.8119 - accuracy: 0.8600
Epoch 4/20
90/90 [==============================] - 5s 59ms/step - loss: 0.7705 - accuracy: 0.8611
Epoch 5/20
90/90 [==============================] - 5s 59ms/step - loss: 0.5174 - accuracy: 0.8867
Epoch 6/20
90/90 [==============================] - 5s 59ms/step - loss: 0.4927 - accuracy: 0.9022
Epoch 7/20
90/90 [==============================] - 5s 59ms/step - loss: 0.5568 - accuracy: 0.8756
Epoch 8/20
90/90 [==============================] - 5s 59ms/step - loss: 0.5912 - accuracy: 0.8833
Epoch 9/20
90/90 [==============================] - 5s 59ms/step - loss: 0.4820 - accuracy: 0.8944
Epoch 10/20
90/90 [==============================] - 5s 59ms/step - loss: 0.4471 - accuracy: 0.8867
Epoch 11/

In [ ]:
mse_test = transfer_model.evaluate(X_test, y_test)

8/8 [==============================] - 1s 76ms/step - loss: 0.9897 - accuracy: 0.8622
